# Order Parameter with Respect to Bilayer Normal

In [713]:
from __future__ import print_function
%matplotlib inline
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
#import scipy.cluster.hierarchy
import scipy.spatial
import math

In [714]:
traj=md.load('a_zero.xtc',top='a_zero_17ns.gro',stride=1)

In [715]:
traj.n_frames

11

In [716]:
name = '0'

In [717]:
topology=traj.topology
DBPCs=topology.select('resid 2233 to 3145') #selected only top leaflet DBPC 
lipidhead=topology.select('resname POPE')
DBPClength=int(52)
numberoflipids=913

In [718]:
len(lipidhead)

47476

In [719]:
913*52

47476

In [720]:
#x,y coordinates of all for finding neighbors to determine local midplane z-value
# here, best to use xy radius of neighbors, since Voronoi across 2 leaflets could have complications

lipidheadxy=list([] for _ in xrange(traj.n_frames))
lipidheadxyz=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(lipidhead)):
        lipidheadi=lipidhead[i]
        lipidheadxy[nn].append(traj.xyz[nn][lipidheadi][0:2:1])
        lipidheadxyz[nn].append(traj.xyz[nn][lipidheadi][0:3:1])


# Random Methods

In [721]:
def find_plane(cord1,cord2,cord3):
    v1 = cord3-cord1
    v2 = cord3-cord2
    cp=np.cross(v1,v2)
    d = cp[0]**2 + cp[1]**2 + cp[2]**2
    x,y,z = cp/(d**.5)
    return [x,y,z]

In [722]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [723]:
def local_vector(cord1,cord2):
    vector = [cord1[0]-cord2[0],cord1[1]-cord2[1],cord1[2]-cord2[2]]
    d=(cord1[0]-cord2[0])**2 + (cord1[1]-cord2[1])**2 + (cord1[2]-cord2[2])**2
    vector=vector/(d**.5)
    return vector
    

In [724]:
def find_plane(cord1,cord2,cord3):
    v1 = cord3-cord1
    v2 = cord3-cord2
    cp=np.cross(v1,v2)
    d = cp[0]**2 + cp[1]**2 + cp[2]**2
    x,y,z = cp/(d**.5)
    return [x,y,z]

In [725]:
def magvect(v):
    magnitude = np.sqrt(v[0]*v[0] + v[1]*v[1] + v[2]*v[2])
    return magnitude

In [726]:
def cos_angle(v1,v2):
    cos = (np.dot(v1,v2)/((magvect(v1)*magvect(v2))))
    return cos

In [727]:
def perpendicular_vector(v):
    if v[1] == 0 and v[2] == 0:
        if v[0] == 0:
            raise ValueError('zero vector')
        else:
            return np.cross(v, [0, 1, 0])
    return np.cross(v, [1, 0, 0])

# Find Membrane Normal

In [728]:
topology=traj.topology
allhead=topology.select('name P8 and resname POPE')

In [729]:
resnames=[atom.residue.name for atom in topology.atoms]

In [730]:
#x,y coordinates of all for finding neighbors to determine local midplane z-value
# here, best to use xy radius of neighbors, since Voronoi across 2 leaflets could have complications

allheadxy=list([] for _ in xrange(traj.n_frames))
allheadz=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(allhead)):
        allheadi=allhead[i]
        allheadxy[nn].append(traj.xyz[nn][allheadi][0:2:1])
        allheadz[nn].append(traj.xyz[nn][allheadi][2])


In [731]:
# mdtraj can't handle neighbors for gro (rather than xtc) inputs, so do it myself
#cutoffsq = 2.4**2; #square of maximum xy distance for neighbors
cutoffsq=1.0**2
neigh=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    neigh[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allheadxy[nn])):
        xyi=allheadxy[nn][i]
        for j in range(len(allhead)-i-1): #not self, but will add for midplane finding
            xyj=allheadxy[nn][j+i+1]
            distsq=(xyi[0]-xyj[0])**2 + (xyi[1]-xyj[1])**2
            if (distsq < cutoffsq):
                neigh[nn][i].append(j+i+1)
                neigh[nn][j+i+1].append(i)
  

In [732]:
#new leaflet id method based on tilt angles; PO4-C4A or ROH-C1 (both +6 beads)
num_head=len(allhead)
tiltvectors=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    tiltvectors[nn]=list([] for _ in xrange(num_head)) #store vector roh-c1 or po4-c4a
    
    for i in range(num_head):
        tiltvectors[nn][i]=traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[i]+6]


In [733]:
# must fix periodic boundary condition errors in tiltvectors, then calculate tiltangle
norms=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    norms[nn]=list([] for _ in xrange(len(allhead)))
    halfx=0.5*traj.unitcell_lengths[nn][0]
    halfy=0.5*traj.unitcell_lengths[nn][1]
    halfz=0.5*traj.unitcell_lengths[nn][2]
    for i in range(len(allhead)):
        norms[nn][i]=np.linalg.norm(tiltvectors[nn][i])
        if (norms[nn][i] > halfz):
            if (np.abs(tiltvectors[nn][i][0]) > halfx):
                if (tiltvectors[nn][i][0]>0): 
                    tiltvectors[nn][i][0]=tiltvectors[nn][i][0]-2*halfx
                else:
                    tiltvectors[nn][i][0]=tiltvectors[nn][i][0]+2*halfx
            if (np.abs(tiltvectors[nn][i][1]) > halfy):
                if (tiltvectors[nn][i][1]>0): 
                    tiltvectors[nn][i][1]=tiltvectors[nn][i][1]-2*halfy
                else:
                    tiltvectors[nn][i][1]=tiltvectors[nn][i][1]+2*halfy
            if (np.abs(tiltvectors[nn][i][2]) > halfz):
                if (tiltvectors[nn][i][2]>0): 
                    tiltvectors[nn][i][2]=tiltvectors[nn][i][2]-2*halfz
                else:
                    tiltvectors[nn][i][2]=tiltvectors[nn][i][2]+2*halfz

    for i in range(len(allhead)):
        norms[nn][i]=np.linalg.norm(tiltvectors[nn][i])


In [734]:
# use neigh to find local average tilt vector, outliers are not in a leaflet
# if pointing neg in z, flip in x,y,z for making average in outer leaflet
localvector=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    localvector[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        localvectorsx=[]
        localvectorsy=[]
        localvectorsz=[]
        sgn=np.sign(tiltvectors[nn][i][2])
        localvectorsx.append(sgn*tiltvectors[nn][i][0]) #include self
        localvectorsy.append(sgn*tiltvectors[nn][i][1]) #include self
        localvectorsz.append(sgn*tiltvectors[nn][i][2]) #include self
        for j in range(len(neigh[nn][i])):
            sgn=np.sign(tiltvectors[nn][neigh[nn][i][j]][2])
            localvectorsx.append(sgn*tiltvectors[nn][neigh[nn][i][j]][0])
            localvectorsy.append(sgn*tiltvectors[nn][neigh[nn][i][j]][1])
            localvectorsz.append(sgn*tiltvectors[nn][neigh[nn][i][j]][2])
        localvector[nn][i]=[np.mean(localvectorsx),np.mean(localvectorsy),np.mean(localvectorsz)]
       

In [735]:
# find angle between orientation vector and local average orientation vector for each lipid
# in range [0,180]
diffangle=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    diffangle[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        normlv=np.linalg.norm(localvector[nn][i])
        normtv=np.linalg.norm(tiltvectors[nn][i])
        cos=np.dot(localvector[nn][i],tiltvectors[nn][i])/(normlv*normtv)
        if (cos==0):
            diffangle[nn][i]=90
        elif (cos==1 and np.sign(localvector[nn][i][2])==np.sign(tiltvectors[nn][i][2])):
            diffangle[nn][i]=0
        elif (cos==1 and np.sign(localvector[nn][i][2])==np.sign(tiltvectors[nn][i][2])):
            diffangle[nn][i]=180
        else:
            diffangle[nn][i]=np.arccos(cos)*180./np.pi


/Users/naveen/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in arccos


In [736]:
#have array with values placing each head in one leaflet: 0=lower, 1=upper, 2=between
head_leaflet=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    head_leaflet[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        if (diffangle[nn][i]>120):
            head_leaflet[nn][i]=0
        elif (diffangle[nn][i]<60):
            head_leaflet[nn][i]=1
        else:
            head_leaflet[nn][i]=2

In [737]:
# want to identify all midplane chol and remove all "midplane" phospholipids
# tilt angle insufficient
# chol: find distance from ROH to nearest PO4; if beyond threshold, then in midplane; 
# else, copy leaflet ID (do for all phospholipids); only if copying non-midplane

for nn in range(traj.n_frames):
    fixedchol=0
    fixedphos=0
    midplanechol=0
    for i in range(len(allhead)):
        mindist=100.0
        if (head_leaflet[nn][i]==2 and (not resnames[allhead[i]]=='CHOL')): #phospholipids, must assign
            for j in range(len(neigh[nn][i])):
                if (not resnames[allhead[neigh[nn][i][j]]]=='CHOL'):
                    dist=np.linalg.norm(traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[neigh[nn][i][j]]])
                    if (dist < mindist and (not head_leaflet[nn][neigh[nn][i][j]]==2)):
                        mindist=dist
                        correct_leaflet=head_leaflet[nn][neigh[nn][i][j]]
            head_leaflet[nn][i]=correct_leaflet
            fixedphos=fixedphos+1
    #must fix all phospholipids before all chols, since some may use fixed phospholipid leaflet id
    # so, must go through allhead twice
    for i in range(len(allhead)):
        mindist=100.0
        if (resnames[allhead[i]]=='CHOL'): #all chol, ignoring angle
            for j in range(len(neigh[nn][i])):
                if (not resnames[allhead[neigh[nn][i][j]]]=='CHOL'):
                    dist=np.linalg.norm(traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[neigh[nn][i][j]]])
                    if (dist < mindist and (not head_leaflet[nn][neigh[nn][i][j]]==2)):
                        mindist=dist
                        correct_leaflet=head_leaflet[nn][neigh[nn][i][j]]
            if (mindist<1.4): #close to a headgroup, so in a leaflet
                head_leaflet[nn][i]=correct_leaflet
                fixedchol=fixedchol+1
            else:
                head_leaflet[nn][i]=2
                midplanechol=midplanechol+1

In [738]:
# only need upper leaflet heads for this situation
lowerheads=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(allhead)):
        if head_leaflet[nn][i]==0:
            lowerheads[nn].append(allhead[i])

In [740]:
#x,y coordinates of outer for Voronoi
headxy=list([] for _ in xrange(traj.n_frames)) #[frame][head]
reg=list([] for _ in xrange(traj.n_frames))
frac_reg=list([] for _ in xrange(traj.n_frames))
vneigh=list([] for _ in xrange(traj.n_frames))
vneighres=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
#    headxy[nn]=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(lowerheads[nn])):
        lowerheadsi=lowerheads[nn][i]
        headxy[nn].append(traj.xyz[nn][lowerheadsi][0:2:1])
        

    vor=scipy.spatial.Voronoi(np.array(headxy[nn]))

#Voronoi neighbors vneigh

#generate list of neighbors, edgeneigh
    edgeneigh=list([] for _ in xrange(len(lowerheads[nn])))
    for k in range(len(vor.ridge_points)):
        pair=vor.ridge_points[k]
        edgeneigh[pair[0]].append(pair[1])
        edgeneigh[pair[1]].append(pair[0])



    vneigh[nn]=list([] for _ in xrange(len(lowerheads[nn])))  #lipid type
    vneighres[nn]=list([] for _ in xrange(len(lowerheads[nn])))
    for i in range(len(lowerheads[nn])):
        vneighheads=edgeneigh[i] #upperhead index, not atom
        if not vneighheads:
            vneigh[nn][i]=[]
            vneighres[nn][i]=[]
        else:
            for j in range(len(vneighheads)):
                vneigh[nn][i].append(resnames[lowerheads[nn][vneighheads[j]]])
                vneighres[nn][i].append(vneighheads[j])


In [741]:
(vneighres[0][0])

[86, 623, 462, 699, 281, 697]

In [742]:
#only care about DBPC's neighbors
DBPC_vneighlist=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    counter=0
    for i in range(len(lowerheads[nn])):
            if resnames[lowerheads[nn][i]]== "POPE":
                counter=counter+1               
                DBPC_vneighlist[nn].append(vneighres[nn][i]) 
for nn in range(traj.n_frames): 
    for i in range(len(DBPC_vneighlist[nn])):
           del DBPC_vneighlist[nn][i][2:]
                

In [743]:
membrane_normal=list([] for x in range(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(DBPC_vneighlist[nn])):
        point1=np.array([allheadxy[nn][i][0],allheadxy[nn][i][1],allheadz[0][i]])
        point2=np.array([allheadxy[nn][DBPC_vneighlist[nn][i][0]][0],allheadxy[nn][DBPC_vneighlist[nn][i][0]][1],allheadz[nn][DBPC_vneighlist[nn][i][0]]])
        point3=np.array([allheadxy[nn][DBPC_vneighlist[nn][i][1]][0],allheadxy[nn][DBPC_vneighlist[nn][i][1]][1],allheadz[nn][DBPC_vneighlist[nn][i][1]]])
        membrane_normal[nn].append(find_plane(point1,point2,point3))

for nn in range(traj.n_frames):
    for i in range(len(membrane_normal[nn])):
        if membrane_normal[nn][i][2]<0:
            membrane_normal[nn][i][0]=(-1)*membrane_normal[nn][i][0]
            membrane_normal[nn][i][1]=(-1)*membrane_normal[nn][i][1]
            membrane_normal[nn][i][2]=(-1)*membrane_normal[nn][i][2]




In [744]:
POPE_in_lowerleaflet = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(lowerheads[nn])):
        if resnames[lowerheads[nn][i]]=='POPE':
            POPE_in_lowerleaflet[nn].append(lowerheads[nn][i])

# Split Desired Lipid into Chains - only need sn1

In [745]:
DBPCchunked=list([] for _ in xrange(traj.n_frames))
sn1=list([] for _ in xrange(traj.n_frames))
sn2=list([] for _ in xrange(traj.n_frames))


for nn in range(traj.n_frames): 
    DBPCchunked[nn]=[DBPCs[i:i + 52] for i in xrange(0, len(DBPCs), 52)]

for nn in range(traj.n_frames):
    for i in range(len(DBPCchunked[nn])):
        if DBPCchunked[nn][i][7] in POPE_in_lowerleaflet[nn]:
            a=DBPCchunked[nn][i][33:50]
            a=np.delete(a,1)
        #b=np.delete(b,1)
            sn1[nn].append(a)
        #sn2[nn].append(b)



# Vector Calculation

In [787]:
localz = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(sn1[nn])):
        
        for j in range(len(sn1[nn][i])-2):
            #print(i,j)
            a=sn1[nn][i][j]-sn1[nn][0][0]+14
            b=sn1[nn][i][j+2]-sn1[nn][0][0]+14
            #print(a,b)
            vector=local_vector(lipidheadxyz[nn][a], lipidheadxyz[nn][b])
            #if vector[0] and vector[1] and vector[2] <0:
                #vector=vector*-1
            localz[nn].append(vector)

In [789]:
localx = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(localz[nn])):
        localx[nn].append(perpendicular_vector(localz[nn][i]))

In [790]:
localy = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(localx[nn])): 
        n = np.cross(localz[nn][i],localx[nn][i])
        n=n/magvect(n)
        if n[0]  <0:
                n=n*-1
        localy[nn].append(n)
     

In [791]:
localx_lipid = list([] for _ in xrange(traj.n_frames))
localy_lipid = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    localx_lipid[nn]=[localx[nn][i:i + 14] for i in xrange(0, len(localx[nn]), 14)]
    localy_lipid[nn]=[localy[nn][i:i + 14] for i in xrange(0, len(localy[nn]), 14)]

In [792]:
for nn in range(traj.n_frames):
    print(len(localx_lipid[nn])==len(sn1[nn]))

True
True
True
True
True
True
True
True
True
True
True


# Order Parameter Calculation

In [793]:
Anglexx = list([] for _ in xrange(traj.n_frames))
Angleyy = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(localx_lipid[nn])):
        for j in range(len(localx_lipid[nn][i])):
            a=cos_angle(localx_lipid[nn][i][j].tolist(),[0,0,1])
            if math.isnan(a)==True:
                print(a)
            if a>np.pi/2:
                a=float(np.pi - a)
            Anglexx[nn].append(a)
            
for nn in range(traj.n_frames):
    for i in range(len(localy_lipid[nn])):
        for j in range(len(localy_lipid[nn][i])):
            a=cos_angle(localy_lipid[nn][i][j],membrane_normal[nn][i])
            #if math.isnan(a)=='True':
                #print(nn,i,j)
            if a>np.pi/2:
                
                a=float(np.pi - a)
            Angleyy[nn].append(a)


In [794]:
Sxx = list([] for _ in xrange(traj.n_frames))
Syy = list([] for _ in xrange(traj.n_frames))
Sxy = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(localx_lipid[nn])):
        for j in range(len(localx_lipid[nn][i])):
            Sxx[nn].append((3*cos_angle(localx_lipid[nn][i][j],membrane_normal[nn][i])**2 - 1)/2)
            Syy[nn].append((3*cos_angle(localy_lipid[nn][i][j],membrane_normal[nn][i])**2 - 1)/2)


In [795]:
for nn in range(traj.n_frames):
    for i in range(len(Anglexx[nn])): 
        if math.isnan(Anglexx[nn][i]):
            print(nn,i)

In [796]:
OrderParam = list([] for _ in xrange(traj.n_frames))
FinalOP = list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(Sxx[nn])):
        OrderParam[nn].append((2*Sxx[nn][i]+Syy[nn][i])/3)
for nn in range(traj.n_frames): 
    FinalOP[nn]=[OrderParam[nn][i:i + 14] for i in xrange(0, len(OrderParam[nn]), 14)]

In [798]:
DataOP = list([] for _ in xrange(traj.n_frames))
Final =[]

for nn in range(traj.n_frames):
    DataOP[nn]=np.mean(FinalOP[nn],axis=0,dtype=float)

Final_AVG=np.mean(DataOP,axis=0,dtype=float)
Final_STD=np.std(DataOP,axis=0,dtype=float)

In [799]:
Final_AVG

array([-0.12620785, -0.1602688 , -0.16824241, -0.18518552, -0.17296974,
       -0.19321158, -0.1522829 , -0.12637643, -0.0715093 , -0.1012805 ,
       -0.14352158, -0.11820879, -0.1258291 , -0.08918895])

In [800]:
file=open('aSymmetric-POPE-OP-AVG.txt',"a+")

file.write(str(float(name)))
file.write(",")

for i in range(len(Final_AVG)):
    file.write(str(float(Final_AVG[i])))
    file.write(",")
               
file.write('\n')
file.write('\n')

In [802]:
file=open('aSymmetric-POPE-OP-STD.txt',"a+")

file.write(str(float(name)))
file.write(",")
for i in range(len(Final_STD)):
    file.write(str(float(Final_STD[i])))
    file.write(",")
file.write('\n')
file.write('\n')

In [ ]:
np.mean(DataOP,axis=0,dtype=float)